# Import Libraries

In [111]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import optuna

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix, classification_report

In [112]:
import warnings
warnings.filterwarnings("ignore")

# Read Data

In [113]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
target = pd.read_csv("target.csv")
submission = pd.read_csv("submission_format.csv")

In [114]:
# Merge train and target
combine = pd.concat([train,target],axis = 1)
df = combine
df

,developerCountry,countryCode,userRatingCount,primaryGenreName,downloads,deviceType,hasPrivacyLink,hasTermsOfServiceLink,hasTermsOfServiceLinkRating,isCorporateEmailScore,adSpent,appAge,averageUserRating,appContentBrandSafetyRating,appDescriptionBrandSafetyRating,mfaRating,coppaRisk
0,NORWAY,RO,127731,Sports,NaN,smartphone,True,True,low,99.0,14.017220,160.400000,4.0,medium,low,low,False
1,ADDRESS NOT LISTED IN PLAYSTORE,GLOBAL,0,Medical,50 - 100,GLOBAL,True,NaN,NaN,99.0,NaN,17.500000,0.0,NaN,low,low,False
2,UNITED ARAB EMIRATES,CZ,51143,Games,50000000 - 100000000,GLOBAL,True,True,low,0.0,31.883163,30.766667,4.0,NaN,low,low,False
3,GERMANY,GLOBAL,1074,Games,NaN,GLOBAL,True,NaN,NaN,99.0,NaN,71.533333,4.0,NaN,low,low,False
4,CANNOT IDENTIFY COUNTRY,GLOBAL,17,Tools,1000 - 5000,GLOBAL,True,NaN,NaN,99.0,NaN,52.400000,4.0,NaN,low,low,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6995,ADDRESS NOT LISTED IN PLAYSTORE,GLOBAL,0,Utilities,NaN,GLOBAL,True,NaN,NaN,99.0,NaN,26.266667,0.0,NaN,low,low,False
6996,ADDRESS NOT LISTED IN PLAYSTORE,GLOBAL,0,Business,NaN,GLOBAL,True,NaN,NaN,NaN,NaN,23.800000,0.0,NaN,low,low,False
6997,ADDRESS NOT LISTED IN PLAYSTORE,GLOBAL,0,Personalization,10 - 50,GLOBAL,True,NaN,NaN,0.0,NaN,27.500000,NaN,NaN,medium,low,False
6998,ADDRESS NOT LISTED IN PLAYSTORE,GLOBAL,0,Business,10 - 50,GLOBAL,True,False,high,99.0,NaN,124.033333,0.0,NaN,low,low,False


In [115]:
# Mengecek jumlah data duplikat
duplikat = df.duplicated()
print("Jumlah duplikasi:", duplikat.sum())

Jumlah duplikasi: 3


In [116]:
test.describe()

,ID,userRatingCount,isCorporateEmailScore,adSpent,appAge,averageUserRating
count,3000.000000,3.000000e+03,2560.000000,574.000000,2983.000000,2486.000000
mean,5016.298333,3.576905e+04,46.444922,286.612276,56.044396,2.490748
std,2894.991128,4.455556e+05,49.415285,1890.997697,36.417410,1.896230
min,1.000000,0.000000e+00,0.000000,0.007450,-1.200000,0.000000
25%,2495.750000,0.000000e+00,0.000000,1.121507,26.933333,0.000000
50%,4969.500000,1.000000e+00,0.000000,7.186537,51.933333,3.000000
75%,7585.500000,8.250000e+01,99.000000,44.878463,77.750000,4.000000
max,9999.000000,1.319941e+07,99.000000,37542.779693,194.466667,5.000000


In [117]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 17 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   ID                               3000 non-null   int64  
 1   developerCountry                 3000 non-null   object 
 2   countryCode                      2971 non-null   object 
 3   userRatingCount                  3000 non-null   int64  
 4   primaryGenreName                 3000 non-null   object 
 5   downloads                        2119 non-null   object 
 6   deviceType                       3000 non-null   object 
 7   hasPrivacyLink                   2684 non-null   object 
 8   hasTermsOfServiceLink            1039 non-null   object 
 9   hasTermsOfServiceLinkRating      1039 non-null   object 
 10  isCorporateEmailScore            2560 non-null   float64
 11  adSpent                          574 non-null    float64
 12  appAge              

In [118]:
df.describe()

,userRatingCount,isCorporateEmailScore,adSpent,appAge,averageUserRating
count,7.000000e+03,5872.000000,1.321000e+03,6950.000000,5768.000000
mean,3.256274e+04,45.740293,3.977563e+03,57.093060,2.482490
std,5.501431e+05,49.361215,9.219919e+04,37.169895,1.905712
min,0.000000e+00,0.000000,7.038525e-03,0.533333,0.000000
25%,0.000000e+00,0.000000,1.086920e+00,27.533333,0.000000
50%,1.000000e+00,0.000000,7.945580e+00,52.866667,3.000000
75%,8.800000e+01,99.000000,4.908375e+01,80.000000,4.000000
max,3.854334e+07,99.000000,3.143906e+06,196.466667,5.000000


In [119]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7000 entries, 0 to 6999
Data columns (total 17 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   developerCountry                 7000 non-null   object 
 1   countryCode                      6936 non-null   object 
 2   userRatingCount                  7000 non-null   int64  
 3   primaryGenreName                 7000 non-null   object 
 4   downloads                        4851 non-null   object 
 5   deviceType                       7000 non-null   object 
 6   hasPrivacyLink                   6250 non-null   object 
 7   hasTermsOfServiceLink            2365 non-null   object 
 8   hasTermsOfServiceLinkRating      2365 non-null   object 
 9   isCorporateEmailScore            5872 non-null   float64
 10  adSpent                          1321 non-null   float64
 11  appAge                           6950 non-null   float64
 12  averageUserRating   

In [120]:
def check_missing_values(df, print_results=True):
    
    # Hitung total missing values per kolom
    missing_values = df.isnull().sum()
    
    # Hitung persentase missing values
    missing_percent = (missing_values / len(df)) * 100
    
    # Buat dataframe hasil
    missing_df = pd.DataFrame({
        'Column': df.columns,
        'Missing_Values': missing_values.values,
        'Missing_Percent': missing_percent.values
    })
    
    # Urutkan dari yang paling banyak missing
    missing_df = missing_df.sort_values('Missing_Percent', ascending=False)
    
    if print_results:
        print("\n=== Missing Values Analysis ===")
        print(f"Total rows in dataset: {len(df)}")
        print("\nMissing values per column:")
        print(missing_df.to_string(index=False))
        
        # Summary
        print("\nSummary:")
        print(f"Columns with missing values: {len(missing_df[missing_df['Missing_Values'] > 0])}/{len(df.columns)}")
        print(f"Columns completely missing: {len(missing_df[missing_df['Missing_Percent'] == 100])}")
        print(f"Most missing column: '{missing_df.iloc[0]['Column']}' with {missing_df.iloc[0]['Missing_Percent']:.2f}% missing")
    
    return missing_df

In [121]:
missing_test = check_missing_values(test)


=== Missing Values Analysis ===
Total rows in dataset: 3000

Missing values per column:
                         Column  Missing_Values  Missing_Percent
    appContentBrandSafetyRating            2671        89.033333
                        adSpent            2426        80.866667
          hasTermsOfServiceLink            1961        65.366667
    hasTermsOfServiceLinkRating            1961        65.366667
                      downloads             881        29.366667
              averageUserRating             514        17.133333
          isCorporateEmailScore             440        14.666667
                 hasPrivacyLink             316        10.533333
                    countryCode              29         0.966667
                         appAge              17         0.566667
appDescriptionBrandSafetyRating               0         0.000000
                             ID               0         0.000000
               developerCountry               0         0.000000
 

In [122]:
missing_info = check_missing_values(df)


=== Missing Values Analysis ===
Total rows in dataset: 7000

Missing values per column:
                         Column  Missing_Values  Missing_Percent
    appContentBrandSafetyRating            6162        88.028571
                        adSpent            5679        81.128571
    hasTermsOfServiceLinkRating            4635        66.214286
          hasTermsOfServiceLink            4635        66.214286
                      downloads            2149        30.700000
              averageUserRating            1232        17.600000
          isCorporateEmailScore            1128        16.114286
                 hasPrivacyLink             750        10.714286
                    countryCode              64         0.914286
                         appAge              50         0.714286
                      mfaRating               0         0.000000
appDescriptionBrandSafetyRating               0         0.000000
               developerCountry               0         0.000000
 

In [123]:
# Correlation Check
selected_cols = df[['isCorporateEmailScore', 'appAge', 'averageUserRating']]
selected_correlation = selected_cols.corr()

selected_correlation

,isCorporateEmailScore,appAge,averageUserRating
isCorporateEmailScore,1.000000,0.123663,-0.107182
appAge,0.123663,1.000000,0.118700
averageUserRating,-0.107182,0.118700,1.000000


# Cleaning Data

In [ ]:
columns_to_drop = [
    'countryCode',
    'appContentBrandSafetyRating',
    'adSpent',
    'isCorporateEmailScore'
]

df_cleaned = df.drop(columns=columns_to_drop)
df_cleaned.columns

Index(['developerCountry', 'userRatingCount', 'primaryGenreName', 'downloads',
       'deviceType', 'hasPrivacyLink', 'hasTermsOfServiceLink',
       'hasTermsOfServiceLinkRating', 'appAge', 'averageUserRating',
       'appDescriptionBrandSafetyRating', 'mfaRating', 'coppaRisk'],
      dtype='object')

In [125]:
columns_to_drop = [
    'countryCode',
    'appContentBrandSafetyRating',
    'adSpent',
    'isCorporateEmailScore',
    'ID'
]
df_test = test.drop(columns = columns_to_drop)
df_test.columns

Index(['developerCountry', 'userRatingCount', 'primaryGenreName', 'downloads',
       'deviceType', 'hasPrivacyLink', 'hasTermsOfServiceLink',
       'hasTermsOfServiceLinkRating', 'appAge', 'averageUserRating',
       'appDescriptionBrandSafetyRating', 'mfaRating'],
      dtype='object')

In [126]:
check_missing_values(df_test)


=== Missing Values Analysis ===
Total rows in dataset: 3000

Missing values per column:
                         Column  Missing_Values  Missing_Percent
          hasTermsOfServiceLink            1961        65.366667
    hasTermsOfServiceLinkRating            1961        65.366667
                      downloads             881        29.366667
              averageUserRating             514        17.133333
                 hasPrivacyLink             316        10.533333
                         appAge              17         0.566667
               developerCountry               0         0.000000
                userRatingCount               0         0.000000
               primaryGenreName               0         0.000000
                     deviceType               0         0.000000
appDescriptionBrandSafetyRating               0         0.000000
                      mfaRating               0         0.000000

Summary:
Columns with missing values: 6/12
Columns completely mis

,Column,Missing_Values,Missing_Percent
6,hasTermsOfServiceLink,1961,65.366667
7,hasTermsOfServiceLinkRating,1961,65.366667
3,downloads,881,29.366667
9,averageUserRating,514,17.133333
5,hasPrivacyLink,316,10.533333
8,appAge,17,0.566667
0,developerCountry,0,0.000000
1,userRatingCount,0,0.000000
2,primaryGenreName,0,0.000000
4,deviceType,0,0.000000


In [127]:
check_missing_values(df_cleaned)


=== Missing Values Analysis ===
Total rows in dataset: 7000

Missing values per column:
                         Column  Missing_Values  Missing_Percent
          hasTermsOfServiceLink            4635        66.214286
    hasTermsOfServiceLinkRating            4635        66.214286
                      downloads            2149        30.700000
              averageUserRating            1232        17.600000
                 hasPrivacyLink             750        10.714286
                         appAge              50         0.714286
               developerCountry               0         0.000000
                userRatingCount               0         0.000000
               primaryGenreName               0         0.000000
                     deviceType               0         0.000000
appDescriptionBrandSafetyRating               0         0.000000
                      mfaRating               0         0.000000
                      coppaRisk               0         0.000000



,Column,Missing_Values,Missing_Percent
6,hasTermsOfServiceLink,4635,66.214286
7,hasTermsOfServiceLinkRating,4635,66.214286
3,downloads,2149,30.700000
9,averageUserRating,1232,17.600000
5,hasPrivacyLink,750,10.714286
8,appAge,50,0.714286
0,developerCountry,0,0.000000
1,userRatingCount,0,0.000000
2,primaryGenreName,0,0.000000
4,deviceType,0,0.000000


In [128]:
df_cleaned

,developerCountry,userRatingCount,primaryGenreName,downloads,deviceType,hasPrivacyLink,hasTermsOfServiceLink,hasTermsOfServiceLinkRating,appAge,averageUserRating,appDescriptionBrandSafetyRating,mfaRating,coppaRisk
0,NORWAY,127731,Sports,NaN,smartphone,True,True,low,160.400000,4.0,low,low,False
1,ADDRESS NOT LISTED IN PLAYSTORE,0,Medical,50 - 100,GLOBAL,True,NaN,NaN,17.500000,0.0,low,low,False
2,UNITED ARAB EMIRATES,51143,Games,50000000 - 100000000,GLOBAL,True,True,low,30.766667,4.0,low,low,False
3,GERMANY,1074,Games,NaN,GLOBAL,True,NaN,NaN,71.533333,4.0,low,low,False
4,CANNOT IDENTIFY COUNTRY,17,Tools,1000 - 5000,GLOBAL,True,NaN,NaN,52.400000,4.0,low,low,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6995,ADDRESS NOT LISTED IN PLAYSTORE,0,Utilities,NaN,GLOBAL,True,NaN,NaN,26.266667,0.0,low,low,False
6996,ADDRESS NOT LISTED IN PLAYSTORE,0,Business,NaN,GLOBAL,True,NaN,NaN,23.800000,0.0,low,low,False
6997,ADDRESS NOT LISTED IN PLAYSTORE,0,Personalization,10 - 50,GLOBAL,True,NaN,NaN,27.500000,NaN,medium,low,False
6998,ADDRESS NOT LISTED IN PLAYSTORE,0,Business,10 - 50,GLOBAL,True,False,high,124.033333,0.0,low,low,False


# Feature Engineering

1. Binning kolom downloads

Karena isi dari kolom downloads itu beragam, maka akan dilakukan pengelompokkan untuk masing-masing jumlah downloads.

In [129]:
# Binning berdasarkan range
def bin_downloads(value):
    if pd.isna(value):
        return np.nan
    value = value.replace(',', '').replace('+', '').strip()
    
    try:
        parts = value.split('-')
        if len(parts) == 2:
            low, high = int(parts[0]), int(parts[1])
            mid = (low + high) / 2
        else:
            mid = int(parts[0])
    except:
        return np.nan

    if 0 <= mid <= 100:
        return 1
    elif 101 <= mid <= 1000:
        return 2
    elif 1001 <= mid <= 10000:
        return 3
    elif 10001 <= mid <= 100000:
        return 4
    elif 100001 <= mid <= 1000000:
        return 5
    elif 1000001 <= mid <= 10000000:
        return 6
    elif 10000001 <= mid <= 100000000:
        return 7
    elif 100000001 <= mid <= 1000000000:
        return 8
    elif mid > 1000000000:
        return 9
    else:
        return np.nan


Imputasi ada 2 kali, yaitu yang pertama melakukan imputasi berdasarkan kolom primaryGenreName. <br> Karena baris yang tidak memiliki data yang cukup, maka akan dilakukan proses imputasi kedua berdasarkan Mode.

In [130]:
df_binned = df_cleaned.copy()

# First Bin
df_binned['downloads_bin'] = df_binned['downloads'].apply(bin_downloads)

# Hitung modus per genre
mode_downloads_bin_by_genre = df_binned.groupby('primaryGenreName')['downloads_bin'].agg(
    lambda x: x.mode().iloc[0] if not x.mode().empty else np.nan
)

# Impute based on Genre
def impute_bin_by_genre(row):
    if pd.isna(row['downloads_bin']):
        return mode_downloads_bin_by_genre.get(row['primaryGenreName'], np.nan)
    return row['downloads_bin']

df_binned['downloads_bin'] = df_binned.apply(impute_bin_by_genre, axis=1)

# Imputation of NaN remainder with overall mode
overall_mode_bin = df_binned['downloads_bin'].mode().iloc[0]
df_binned['downloads_bin'] = df_binned['downloads_bin'].fillna(overall_mode_bin)

In [131]:
df_binned_test = df_test.copy()

# First Bin
df_binned_test['downloads_bin'] = df_binned_test['downloads'].apply(bin_downloads)

# Hitung modus per genre
mode_downloads_bin_by_genre_test = df_binned_test.groupby('primaryGenreName')['downloads_bin'].agg(
    lambda x: x.mode().iloc[0] if not x.mode().empty else np.nan
)

# Impute based on Genre
def impute_bin_by_genre_test(row):
    if pd.isna(row['downloads_bin']):
        return mode_downloads_bin_by_genre_test.get(row['primaryGenreName'], np.nan)
    return row['downloads_bin']

df_binned_test['downloads_bin'] = df_binned_test.apply(impute_bin_by_genre_test, axis=1)

# Imputation of NaN remainder with overall mode
overall_mode_bin_test = df_binned_test['downloads_bin'].mode().iloc[0]
df_binned_test['downloads_bin'] = df_binned_test['downloads_bin'].fillna(overall_mode_bin_test)

In [132]:
df_binned_test

,developerCountry,userRatingCount,primaryGenreName,downloads,deviceType,hasPrivacyLink,hasTermsOfServiceLink,hasTermsOfServiceLinkRating,appAge,averageUserRating,appDescriptionBrandSafetyRating,mfaRating,downloads_bin
0,ADDRESS NOT LISTED IN PLAYSTORE,2,Medical,NaN,GLOBAL,True,NaN,NaN,81.600000,5.0,low,low,1.0
1,ICELAND,0,Games,5000 - 10000,GLOBAL,True,True,low,24.700000,0.0,low,low,3.0
2,UNITED STATES,13059,Games,NaN,smartphone,True,False,high,40.300000,4.0,low,low,6.0
3,ADDRESS NOT LISTED IN PLAYSTORE,0,Games,NaN,GLOBAL,True,False,high,39.233333,0.0,medium,low,6.0
4,ADDRESS NOT LISTED IN PLAYSTORE,0,Shopping,NaN,GLOBAL,NaN,NaN,NaN,65.533333,0.0,low,low,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,UNITED STATES,0,Productivity,NaN,GLOBAL,True,True,low,94.100000,0.0,low,low,1.0
2996,CANNOT IDENTIFY COUNTRY,10,Business,100 - 500,GLOBAL,True,NaN,NaN,25.033333,5.0,low,low,2.0
2997,ADDRESS NOT LISTED IN PLAYSTORE,2,Finance,NaN,GLOBAL,NaN,NaN,NaN,76.000000,2.0,low,low,1.0
2998,ADDRESS NOT LISTED IN PLAYSTORE,0,Business,1 - 5,GLOBAL,True,NaN,NaN,100.633333,0.0,low,low,1.0


Drop kolom downloads

2. Handle Missing Value dari kolom hasPrivacyLink

In [133]:
# Handle Missing value for 'hasPrivacyLink' col
df_binned['hasPrivacyLink'] = df_binned['hasPrivacyLink'].fillna(df_binned['hasPrivacyLink'].mode()[0])
df_binned_test['hasPrivacyLink'] = df_binned_test['hasPrivacyLink'].fillna(df_binned_test['hasPrivacyLink'].mode()[0])

3. Feature Eng untuk kolom userRatingCount

Karena userRatingCount tidak begitu jelas, jadi kita permudah menjadi apakah aplikasi tersebut memiliki rating atau tidak.

In [134]:
# Split as category for 'userRatingCount' col
df_binned['hasRating'] = df_binned['userRatingCount'] > 0
df_binned_test['hasRating'] = df_binned_test['userRatingCount'] > 0

4. Feature Engineering kolom appAge

Handle Missing Value

In [135]:
# Handle Missing Value appAge
median_appAge = df['appAge'].median()
df_binned['appAge'].fillna(median_appAge, inplace=True)
df_binned_test['appAge'].fillna(median_appAge, inplace=True)

Karena kolom appAge isinya berupa angka, maka akan kita kelompokkan sesuai dengan usia aplikasi itu sendiri. Rule based-nya ada di bawah

Binning Kolom appAge

In [136]:
# Binning for appAge
def age_bin(days):
    if pd.isna(days):
        return np.nan
    elif days <= 90:
        return 'Baru'
    elif days <= 365:
        return 'Menengah'
    else:
        return 'Lama'

df_binned['appAge_binned'] = df_binned['appAge'].apply(age_bin)
df_binned_test['appAge_binned'] = df_binned_test['appAge'].apply(age_bin)

5. Buat kolom adSpent baru berdasarkan mfaRating dan averageUserRating

Karena terdapat banyak missing value pada adSpent, maka akan dilakukan imputasi atau membuat kolom baru berdasarkan hubungan antara mfaRating dan averageUserRating.

In [137]:
# Handle Missing Value untuk averageUserRating
df_binned['averageUserRating'] = df_binned['averageUserRating'].fillna(df_binned['averageUserRating'].median())
df_binned_test['averageUserRating'] = df_binned_test['averageUserRating'].fillna(df_binned_test['averageUserRating'].median())

In [138]:
# Predict adSpent_pred
def predict_ad_spent_clean(row):
    rating = row['averageUserRating']
    mfa = row['mfaRating']

    if pd.isna(mfa):
        return np.nan

    if rating >= 4 and mfa == 'low':
        return 'high'
    elif rating >= 4 and mfa in ['medium', 'high']:
        return 'medium'
    elif rating < 4 and mfa == 'low':
        return 'medium'
    elif rating < 4 and mfa in ['medium', 'high']:
        return 'low'
    elif rating == 0 and mfa == 'low':
        return 'low'
    else:
        return 'unknown'

In [139]:
df_binned['adSpent_pred'] = df_binned.apply(predict_ad_spent_clean, axis=1)
df_binned_test['adSpent_pred'] = df_binned_test.apply(predict_ad_spent_clean, axis=1)

5. Binning for devCountry

Karena kolom devCountry banyak yang tidak mengisi asal negaranya, maka kita buat kolom baru, yaitu isCountryExist. Kolom ini dibuat berdasarkan kolom devCountry yang bertujuan untuk memberitahu bahwa apakah developer menyantumkan negara atau tidak.

In [140]:
import pycountry
valid_countries = {country.name.upper() for country in pycountry.countries}

# Fungsi untuk mengecek validitas negara
def check_country_exist(country_name):
    if pd.isna(country_name):
        return False
    return country_name.strip().upper() in valid_countries

In [141]:
df_binned['isCountryExist'] = df_binned['developerCountry'].apply(check_country_exist)
df_binned_test['isCountryExist'] = df_binned_test['developerCountry'].apply(check_country_exist)

6. Handle kolom primaryGenreName

Karena genrenya terlalu banyak, jadi akan dilakukan pengelompokkan untuk masing-masing genre.

In [142]:
# Dictionary untuk binning genre
genre_binning = {
    # Hiburan
    'Games': 'Hiburan',
    'Entertainment': 'Hiburan',
    'Music & Audio': 'Hiburan',
    'Music': 'Hiburan',
    'Social': 'Hiburan',
    'Social Networking': 'Hiburan',
    'Video Players & Editors': 'Hiburan',
    'Photo & Video': 'Hiburan',
    'Comics': 'Hiburan',

    # Gaya Hidup
    'Lifestyle': 'Gaya Hidup',
    'Food & Drink': 'Gaya Hidup',
    'Shopping': 'Gaya Hidup',
    'Travel & Local': 'Gaya Hidup',
    'Travel': 'Gaya Hidup',
    'Maps & Navigation': 'Gaya Hidup',
    'Sports': 'Gaya Hidup',
    'Auto & Vehicles': 'Gaya Hidup',
    'Art & Design': 'Gaya Hidup',
    'House & Home': 'Gaya Hidup',
    'Weather': 'Gaya Hidup',
    'Events': 'Gaya Hidup',
    'Beauty': 'Gaya Hidup',
    'Dating': 'Gaya Hidup',

    # Produktivitas
    'Productivity': 'Produktivitas',
    'Tools': 'Produktivitas',
    'Utilities': 'Produktivitas',
    'Finance': 'Produktivitas',
    'Personalization': 'Produktivitas',
    'Communication': 'Produktivitas',
    'News & Magazines': 'Produktivitas',
    'News': 'Produktivitas',
    'Reference': 'Produktivitas',
    'Book': 'Produktivitas',
    'Navigation': 'Produktivitas',
    'Magazines & Newspapers': 'Produktivitas',

    # Informasi & Pendidikan
    'Education': 'Informasi & Pendidikan',
    'Books & Reference': 'Informasi & Pendidikan',
    'Medical': 'Informasi & Pendidikan',
    'Photography': 'Informasi & Pendidikan',
    'Science & Technology': 'Informasi & Pendidikan',
    'Developer Tools': 'Informasi & Pendidikan',

    # Bisnis & Pengembangan
    'Business': 'Bisnis & Pengembangan',

    # Kesehatan & Keluarga
    'Health & Fitness': 'Kesehatan & Keluarga',
    'Parenting': 'Kesehatan & Keluarga',
    'Libraries & Demo': 'Kesehatan & Keluarga'
}

# Imputasi nilai NaN dengan kategori 'Lainnya'
df_binned['primaryGenreName'] = df_binned['primaryGenreName'].fillna('Lainnya')
df_binned_test['primaryGenreName'] = df_binned_test['primaryGenreName'].fillna('Lainnya')

# Binning genre
df_binned['genre_binned'] = df_binned['primaryGenreName'].map(genre_binning).fillna('Lainnya')
df_binned_test['genre_binned'] = df_binned_test['primaryGenreName'].map(genre_binning).fillna('Lainnya')

7. Buat kolom appContentBrandSafetyRating baru berdasarkan mfaRating, averageUserRating, appDescriptionBrandSafetyRating, dan adSpent_pred

In [143]:
# Define rule-based function to predict appContentBrandSafetyRating
def predict_content_brand_safety(row):
    desc_rating = row['appDescriptionBrandSafetyRating']
    mfa = row['mfaRating']
    ad_spent = row['adSpent_pred']
    rating = row['averageUserRating']

    if desc_rating == 'high' and mfa == 'low' and ad_spent == 'high' and rating >= 4:
        return 'high'
    elif desc_rating == 'low' and (mfa == 'high' or ad_spent == 'low') and rating <= 2.5:
        return 'low'
    elif desc_rating == 'medium' and mfa == 'medium' and ad_spent == 'medium':
        return 'medium'
    elif desc_rating == 'high' and mfa == 'medium' and ad_spent in ['medium', 'high']:
        return 'high'
    elif desc_rating == 'medium' and mfa == 'low' and ad_spent == 'high' and rating > 3.5:
        return 'high'
    else:
        return 'medium'

In [144]:
df_binned['appContentBrandSafetyRating_pred'] = df_binned.apply(predict_content_brand_safety, axis=1)
df_binned_test['appContentBrandSafetyRating_pred'] = df_binned_test.apply(predict_content_brand_safety, axis=1)

In [145]:
df_binned

,developerCountry,userRatingCount,primaryGenreName,downloads,deviceType,hasPrivacyLink,hasTermsOfServiceLink,hasTermsOfServiceLinkRating,appAge,averageUserRating,appDescriptionBrandSafetyRating,mfaRating,coppaRisk,downloads_bin,hasRating,appAge_binned,adSpent_pred,isCountryExist,genre_binned,appContentBrandSafetyRating_pred
0,NORWAY,127731,Sports,NaN,smartphone,True,True,low,160.400000,4.0,low,low,False,1.0,True,Menengah,high,True,Gaya Hidup,medium
1,ADDRESS NOT LISTED IN PLAYSTORE,0,Medical,50 - 100,GLOBAL,True,NaN,NaN,17.500000,0.0,low,low,False,1.0,False,Baru,medium,False,Informasi & Pendidikan,medium
2,UNITED ARAB EMIRATES,51143,Games,50000000 - 100000000,GLOBAL,True,True,low,30.766667,4.0,low,low,False,7.0,True,Baru,high,True,Hiburan,medium
3,GERMANY,1074,Games,NaN,GLOBAL,True,NaN,NaN,71.533333,4.0,low,low,False,6.0,True,Baru,high,True,Hiburan,medium
4,CANNOT IDENTIFY COUNTRY,17,Tools,1000 - 5000,GLOBAL,True,NaN,NaN,52.400000,4.0,low,low,False,3.0,True,Baru,high,False,Produktivitas,medium
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6995,ADDRESS NOT LISTED IN PLAYSTORE,0,Utilities,NaN,GLOBAL,True,NaN,NaN,26.266667,0.0,low,low,False,1.0,False,Baru,medium,False,Produktivitas,medium
6996,ADDRESS NOT LISTED IN PLAYSTORE,0,Business,NaN,GLOBAL,True,NaN,NaN,23.800000,0.0,low,low,False,1.0,False,Baru,medium,False,Bisnis & Pengembangan,medium
6997,ADDRESS NOT LISTED IN PLAYSTORE,0,Personalization,10 - 50,GLOBAL,True,NaN,NaN,27.500000,3.0,medium,low,False,1.0,False,Baru,medium,False,Produktivitas,medium
6998,ADDRESS NOT LISTED IN PLAYSTORE,0,Business,10 - 50,GLOBAL,True,False,high,124.033333,0.0,low,low,False,1.0,False,Menengah,medium,False,Bisnis & Pengembangan,medium


# Data Preprocessing

In [146]:
# Ubah semua kolom boolean menjadi integer
bool_cols = df_binned.select_dtypes(include='bool').columns
df_binned[bool_cols] = df_binned[bool_cols].astype(int)

In [147]:
bool_test_cols = df_binned_test.select_dtypes(include='bool').columns
df_binned_test[bool_test_cols] = df_binned_test[bool_test_cols].astype(int)

In [148]:
df_binned

,developerCountry,userRatingCount,primaryGenreName,downloads,deviceType,hasPrivacyLink,hasTermsOfServiceLink,hasTermsOfServiceLinkRating,appAge,averageUserRating,appDescriptionBrandSafetyRating,mfaRating,coppaRisk,downloads_bin,hasRating,appAge_binned,adSpent_pred,isCountryExist,genre_binned,appContentBrandSafetyRating_pred
0,NORWAY,127731,Sports,NaN,smartphone,1,True,low,160.400000,4.0,low,low,0,1.0,1,Menengah,high,1,Gaya Hidup,medium
1,ADDRESS NOT LISTED IN PLAYSTORE,0,Medical,50 - 100,GLOBAL,1,NaN,NaN,17.500000,0.0,low,low,0,1.0,0,Baru,medium,0,Informasi & Pendidikan,medium
2,UNITED ARAB EMIRATES,51143,Games,50000000 - 100000000,GLOBAL,1,True,low,30.766667,4.0,low,low,0,7.0,1,Baru,high,1,Hiburan,medium
3,GERMANY,1074,Games,NaN,GLOBAL,1,NaN,NaN,71.533333,4.0,low,low,0,6.0,1,Baru,high,1,Hiburan,medium
4,CANNOT IDENTIFY COUNTRY,17,Tools,1000 - 5000,GLOBAL,1,NaN,NaN,52.400000,4.0,low,low,0,3.0,1,Baru,high,0,Produktivitas,medium
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6995,ADDRESS NOT LISTED IN PLAYSTORE,0,Utilities,NaN,GLOBAL,1,NaN,NaN,26.266667,0.0,low,low,0,1.0,0,Baru,medium,0,Produktivitas,medium
6996,ADDRESS NOT LISTED IN PLAYSTORE,0,Business,NaN,GLOBAL,1,NaN,NaN,23.800000,0.0,low,low,0,1.0,0,Baru,medium,0,Bisnis & Pengembangan,medium
6997,ADDRESS NOT LISTED IN PLAYSTORE,0,Personalization,10 - 50,GLOBAL,1,NaN,NaN,27.500000,3.0,medium,low,0,1.0,0,Baru,medium,0,Produktivitas,medium
6998,ADDRESS NOT LISTED IN PLAYSTORE,0,Business,10 - 50,GLOBAL,1,False,high,124.033333,0.0,low,low,0,1.0,0,Menengah,medium,0,Bisnis & Pengembangan,medium


In [ ]:
class DataEncoder:
    def __init__(self):
        self.label_encoders = {}
        self.onehot_encoders = {}
        self.onehot_columns = []

    def fit(self, df, label_cols=[], onehot_cols=[]):
        # Fit LabelEncoders
        for col in label_cols:
            le = LabelEncoder()
            df[col] = df[col].fillna("Unknown")
            le.fit(df[col])
            self.label_encoders[col] = le

        # Fit OneHotEncoders
        for col in onehot_cols:
            ohe = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
            df[col] = df[col].fillna("undetermined")
            ohe.fit(df[[col]])
            self.onehot_encoders[col] = ohe
            self.onehot_columns.append(col)

    def transform(self, df):
        df_transformed = df.copy()

        # Apply label encoders
        for col, le in self.label_encoders.items():
            df_transformed[col] = df_transformed[col].fillna("Unknown")
            df_transformed[f"{col}_encoded"] = le.transform(df_transformed[col])

        # Apply one-hot encoders
        for col in self.onehot_columns:
            df_transformed[col] = df_transformed[col].fillna("undetermined")
            ohe = self.onehot_encoders[col]
            ohe_array = ohe.transform(df_transformed[[col]])
            ohe_cols = ohe.get_feature_names_out([col])
            ohe_df = pd.DataFrame(ohe_array, columns=ohe_cols, index=df_transformed.index)
            df_transformed = pd.concat([df_transformed, ohe_df], axis=1)

        return df_transformed

    def fit_transform(self, df, label_cols=[], onehot_cols=[]):
        self.fit(df, label_cols=label_cols, onehot_cols=onehot_cols)
        return self.transform(df)

In [150]:
encoder = DataEncoder()

# Define kolom yang ingin di-encode
label_cols = ['appDescriptionBrandSafetyRating', 'mfaRating', 'adSpent_pred','appContentBrandSafetyRating_pred','appAge_binned']
onehot_cols = ['deviceType','genre_binned']

# Fit dan transform data training
df_encoded = encoder.fit_transform(df_binned, label_cols=label_cols, onehot_cols=onehot_cols)

# Transform data test pakai encoder yang sudah fit
df_encoded_test = encoder.transform(df_binned_test)


In [151]:
dt_train = df_encoded.drop(columns=['developerCountry',
                                    'userRatingCount',
                                    'primaryGenreName',
                                    'downloads',
                                    'deviceType',
                                    'appAge',
                                    'genre_binned'])

In [152]:
dt_train

,hasPrivacyLink,hasTermsOfServiceLink,hasTermsOfServiceLinkRating,averageUserRating,appDescriptionBrandSafetyRating,mfaRating,coppaRisk,downloads_bin,hasRating,appAge_binned,...,deviceType_smartphone,deviceType_tablet,deviceType_undetermined,genre_binned_Bisnis & Pengembangan,genre_binned_Gaya Hidup,genre_binned_Hiburan,genre_binned_Informasi & Pendidikan,genre_binned_Kesehatan & Keluarga,genre_binned_Lainnya,genre_binned_Produktivitas
0,1,True,low,4.0,low,low,0,1.0,1,Menengah,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,1,NaN,NaN,0.0,low,low,0,1.0,0,Baru,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,1,True,low,4.0,low,low,0,7.0,1,Baru,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,1,NaN,NaN,4.0,low,low,0,6.0,1,Baru,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,1,NaN,NaN,4.0,low,low,0,3.0,1,Baru,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6995,1,NaN,NaN,0.0,low,low,0,1.0,0,Baru,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
6996,1,NaN,NaN,0.0,low,low,0,1.0,0,Baru,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
6997,1,NaN,NaN,3.0,medium,low,0,1.0,0,Baru,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
6998,1,False,high,0.0,low,low,0,1.0,0,Menengah,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [153]:
miss = check_missing_values(dt_train)


=== Missing Values Analysis ===
Total rows in dataset: 7000

Missing values per column:
                                  Column  Missing_Values  Missing_Percent
             hasTermsOfServiceLinkRating            4635        66.214286
                   hasTermsOfServiceLink            4635        66.214286
                          hasPrivacyLink               0         0.000000
appContentBrandSafetyRating_pred_encoded               0         0.000000
                    genre_binned_Lainnya               0         0.000000
       genre_binned_Kesehatan & Keluarga               0         0.000000
     genre_binned_Informasi & Pendidikan               0         0.000000
                    genre_binned_Hiburan               0         0.000000
                 genre_binned_Gaya Hidup               0         0.000000
      genre_binned_Bisnis & Pengembangan               0         0.000000
                 deviceType_undetermined               0         0.000000
                       

In [154]:
dt_val = df_encoded_test.drop(columns=['developerCountry',
                                    'userRatingCount',
                                    'primaryGenreName',
                                    'downloads',
                                    'deviceType',
                                    'appAge',
                                    'genre_binned'])

In [155]:
miss = check_missing_values(dt_val)


=== Missing Values Analysis ===
Total rows in dataset: 3000

Missing values per column:
                                  Column  Missing_Values  Missing_Percent
             hasTermsOfServiceLinkRating            1961        65.366667
                   hasTermsOfServiceLink            1961        65.366667
                          hasPrivacyLink               0         0.000000
                   appAge_binned_encoded               0         0.000000
                    genre_binned_Lainnya               0         0.000000
       genre_binned_Kesehatan & Keluarga               0         0.000000
     genre_binned_Informasi & Pendidikan               0         0.000000
                    genre_binned_Hiburan               0         0.000000
                 genre_binned_Gaya Hidup               0         0.000000
      genre_binned_Bisnis & Pengembangan               0         0.000000
                 deviceType_undetermined               0         0.000000
                       

In [156]:
dt_valid = pd.concat([dt_val,submission], axis = 1) 
dt_valid

,hasPrivacyLink,hasTermsOfServiceLink,hasTermsOfServiceLinkRating,averageUserRating,appDescriptionBrandSafetyRating,mfaRating,downloads_bin,hasRating,appAge_binned,adSpent_pred,...,deviceType_undetermined,genre_binned_Bisnis & Pengembangan,genre_binned_Gaya Hidup,genre_binned_Hiburan,genre_binned_Informasi & Pendidikan,genre_binned_Kesehatan & Keluarga,genre_binned_Lainnya,genre_binned_Produktivitas,ID,coppaRisk
0,1,NaN,NaN,5.0,low,low,1.0,1,Baru,high,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,2807,NaN
1,1,True,low,0.0,low,low,3.0,0,Baru,medium,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1742,NaN
2,1,False,high,4.0,low,low,6.0,1,Baru,high,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,806,NaN
3,1,False,high,0.0,medium,low,6.0,0,Baru,medium,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,2635,NaN
4,1,NaN,NaN,0.0,low,low,1.0,0,Baru,medium,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,9047,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,1,True,low,0.0,low,low,1.0,0,Menengah,medium,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2288,NaN
2996,1,NaN,NaN,5.0,low,low,2.0,1,Baru,high,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,5541,NaN
2997,1,NaN,NaN,2.0,low,low,1.0,1,Baru,medium,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,9259,NaN
2998,1,NaN,NaN,0.0,low,low,1.0,0,Menengah,medium,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,3477,NaN


# Handle Imbalance Data

In [157]:
dt = pd.read_csv("train - bismillah_udah_fix.csv")
dval = pd.read_csv("test - bismillah_udah_fix.csv")

In [161]:
aa = pd.read_csv('train_encoded_with_NaN.csv')
aa.columns

Index(['hasPrivacyLink', 'hasTermsOfServiceLink',
       'hasTermsOfServiceLinkRating', 'averageUserRating',
       'appDescriptionBrandSafetyRating', 'mfaRating', 'coppaRisk',
       'downloads_bin', 'hasRating', 'appAge_binned', 'adSpent_pred',
       'isCountryExist', 'appContentBrandSafetyRating_pred',
       'appDescriptionBrandSafetyRating_encoded', 'mfaRating_encoded',
       'adSpent_pred_encoded', 'appContentBrandSafetyRating_pred_encoded',
       'appAge_binned_encoded', 'deviceType_GLOBAL',
       'deviceType_connected-tv/ott', 'deviceType_smartphone',
       'deviceType_tablet', 'deviceType_undetermined',
       'genre_binned_Bisnis & Pengembangan', 'genre_binned_Gaya Hidup',
       'genre_binned_Hiburan', 'genre_binned_Informasi & Pendidikan',
       'genre_binned_Kesehatan & Keluarga', 'genre_binned_Lainnya',
       'genre_binned_Produktivitas'],
      dtype='object')

In [162]:
aas = check_missing_values(aa)


=== Missing Values Analysis ===
Total rows in dataset: 7000

Missing values per column:
                                  Column  Missing_Values  Missing_Percent
             hasTermsOfServiceLinkRating            4635        66.214286
                   hasTermsOfServiceLink            4635        66.214286
                          hasPrivacyLink               0         0.000000
appContentBrandSafetyRating_pred_encoded               0         0.000000
                    genre_binned_Lainnya               0         0.000000
       genre_binned_Kesehatan & Keluarga               0         0.000000
     genre_binned_Informasi & Pendidikan               0         0.000000
                    genre_binned_Hiburan               0         0.000000
                 genre_binned_Gaya Hidup               0         0.000000
      genre_binned_Bisnis & Pengembangan               0         0.000000
                 deviceType_undetermined               0         0.000000
                       

Encode yang belum kena encode

In [159]:
# For Training
categorical_cols = [
    'appDescriptionBrandSafetyRating',
    'hasTermsOfServiceLinkRating',
    'hasTermsOfServiceLink',
    'mfaRating',
    'appAge_binned',
    'adSpent_pred',
    'appContentBrandSafetyRating_pred'
]

dt_train_enc = aa.copy()
le_dict = {}

for col in categorical_cols:
    le = LabelEncoder()
    dt_train_enc[col] = le.fit_transform(dt_train_enc[col])
    le_dict[col] = le

In [160]:
oo = check_missing_values(dt_train_enc)


=== Missing Values Analysis ===
Total rows in dataset: 7000

Missing values per column:
                                  Column  Missing_Values  Missing_Percent
                          hasPrivacyLink               0              0.0
                   hasTermsOfServiceLink               0              0.0
                    genre_binned_Lainnya               0              0.0
       genre_binned_Kesehatan & Keluarga               0              0.0
     genre_binned_Informasi & Pendidikan               0              0.0
                    genre_binned_Hiburan               0              0.0
                 genre_binned_Gaya Hidup               0              0.0
      genre_binned_Bisnis & Pengembangan               0              0.0
                 deviceType_undetermined               0              0.0
                       deviceType_tablet               0              0.0
                   deviceType_smartphone               0              0.0
             deviceType

In [1755]:
# For Test
categorical_cols = [
    'appDescriptionBrandSafetyRating',
    'mfaRating',
    'appAge_binned',
    'adSpent_pred',
    'appContentBrandSafetyRating_pred'
]

dt_val_enc = dval.copy()
le_dict = {}

for col in categorical_cols:
    le = LabelEncoder()
    dt_val_enc[col] = le.fit_transform(dt_val_enc[col])
    le_dict[col] = le

In [1756]:
dt_val_enc.columns

Index(['hasPrivacyLink', 'averageUserRating',
       'appDescriptionBrandSafetyRating', 'mfaRating', 'downloads_bin',
       'hasRating', 'appAge_binned', 'adSpent_pred', 'isCountryExist',
       'appContentBrandSafetyRating_pred',
       'appDescriptionBrandSafetyRating_encoded', 'mfaRating_encoded',
       'adSpent_pred_encoded', 'appContentBrandSafetyRating_pred_encoded',
       'deviceType_GLOBAL', 'deviceType_connected-tv/ott',
       'deviceType_smartphone', 'deviceType_tablet', 'deviceType_undetermined',
       'genre_binned_Bisnis & Pengembangan', 'genre_binned_Gaya Hidup',
       'genre_binned_Hiburan', 'genre_binned_Informasi & Pendidikan',
       'genre_binned_Kesehatan & Keluarga', 'genre_binned_Lainnya',
       'genre_binned_Produktivitas', 'ID', 'coppaRisk'],
      dtype='object')

In [ ]:
# Splitting Dataset for Training
X = dt_train_enc.drop(columns = ['coppaRisk'])
y = dt_train_enc['coppaRisk']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

X_train.shape, X_test.shape,y_train.shape,y_test.shape

((5600, 26), (1400, 26), (5600,), (1400,))

1. Random Forest with class_weight = 'balanced'

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(class_weight='balanced', random_state=42)
rf.fit(X_train, y_train)
rf_probs = rf.predict_proba(X_test)[:, 1]
rf_auc = roc_auc_score(y_test, rf_probs)

rf_auc  

0.8222177214612132

In [1737]:
import optuna
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier

def objective(trial):
    n_estimators = trial.suggest_int("n_estimators", 100, 500)
    max_depth = trial.suggest_int("max_depth", 5, 50)
    min_samples_split = trial.suggest_int("min_samples_split", 2, 10)
    min_samples_leaf = trial.suggest_int("min_samples_leaf", 1, 10)
    max_features = trial.suggest_categorical("max_features", ["sqrt", "log2", None])
    bootstrap = trial.suggest_categorical("bootstrap", [True, False])
    criterion = trial.suggest_categorical("criterion", ["gini", "entropy", "log_loss"])

    rf = RandomForestClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        min_samples_split=min_samples_split,
        min_samples_leaf=min_samples_leaf,
        max_features=max_features,
        bootstrap=bootstrap,
        criterion=criterion,
        class_weight='balanced',
        random_state=42,
        n_jobs=-1
    )
    
    score = cross_val_score(rf, X_train, y_train, cv=10, scoring='roc_auc')
    return score.mean()


In [1738]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=100)

# Train final model with best params
best_params = study.best_params
best_rf = RandomForestClassifier(
    **best_params, class_weight='balanced', random_state=42, n_jobs=-1
)
best_rf.fit(X_train, y_train)
best_rf_auc = roc_auc_score(y_test, best_rf.predict_proba(X_test)[:, 1])

print("Best Params:", best_params)
print("Best AUC:", best_rf_auc)

[I 2025-04-12 23:48:41,608] A new study created in memory with name: no-name-270d4d94-5568-4399-bb40-ef690847a373
[I 2025-04-12 23:48:49,591] Trial 0 finished with value: 0.8240351216244074 and parameters: {'n_estimators': 476, 'max_depth': 33, 'min_samples_split': 2, 'min_samples_leaf': 2, 'max_features': None, 'bootstrap': True, 'criterion': 'gini'}. Best is trial 0 with value: 0.8240351216244074.
[I 2025-04-12 23:48:52,464] Trial 1 finished with value: 0.8469676954302573 and parameters: {'n_estimators': 238, 'max_depth': 18, 'min_samples_split': 5, 'min_samples_leaf': 4, 'max_features': 'log2', 'bootstrap': False, 'criterion': 'entropy'}. Best is trial 1 with value: 0.8469676954302573.
[I 2025-04-12 23:48:56,033] Trial 2 finished with value: 0.824438757842621 and parameters: {'n_estimators': 221, 'max_depth': 40, 'min_samples_split': 10, 'min_samples_leaf': 1, 'max_features': None, 'bootstrap': True, 'criterion': 'entropy'}. Best is trial 1 with value: 0.8469676954302573.
[I 2025-04

Best Params: {'n_estimators': 321, 'max_depth': 37, 'min_samples_split': 8, 'min_samples_leaf': 9, 'max_features': 'sqrt', 'bootstrap': True, 'criterion': 'log_loss'}
Best AUC: 0.8582146178378471


In [1740]:
best_params

{'n_estimators': 321,
 'max_depth': 37,
 'min_samples_split': 8,
 'min_samples_leaf': 9,
 'max_features': 'sqrt',
 'bootstrap': True,
 'criterion': 'log_loss'}

In [1741]:
import pickle
best_params_rf = best_params

with open("best_params_rf_tune_bawangPutih.pkl", "wb") as file:
    pickle.dump(best_params_rf, file)

print("✅ Save Params Succeed!")

✅ Save Params Succeed!


Uji Model Menggunakan Data Validasi

In [ ]:
cols_to_drop = ['coppaRisk','ID']
dt_val_pred = dt_val_enc.drop(columns=[col for col in cols_to_drop if col in dt_val_enc.columns])

# 4. Samakan urutan kolom
dt_val_pred = dt_val_pred[X.columns]

# 5. Prediksi label dan probabilitas
submission['coppaRisk_label'] = best_rf.predict(dt_val_pred)

# 6. Simpan ke file
submission.to_csv("hasil_prediksi_rf.csv", index=False)

# 7. Cek hasil
print(submission.head())

     ID  coppaRisk  coppaRisk_label
0  2807        NaN                1
1  1742        NaN                1
2   806        NaN                1
3  2635        NaN                0
4  9047        NaN                0


In [1768]:
subm = pd.read_csv('hasil_prediksi_rf.csv')
subm

,ID,coppaRisk,coppaRisk_label
0,2807,NaN,1
1,1742,NaN,1
2,806,NaN,1
3,2635,NaN,0
4,9047,NaN,0
...,...,...,...
2995,2288,NaN,0
2996,5541,NaN,0
2997,9259,NaN,0
2998,3477,NaN,0


In [1769]:
subm = subm.drop(columns='coppaRisk')
subm

,ID,coppaRisk_label
0,2807,1
1,1742,1
2,806,1
3,2635,0
4,9047,0
...,...,...
2995,2288,0
2996,5541,0
2997,9259,0
2998,3477,0


In [1770]:
subm.rename(columns={
    'coppaRisk_label': 'coppaRisk'
}, inplace=True)


In [1766]:
subm.to_csv('rf_fineTune.csv', index = False)